# Práctica 1: Criptografía clásica
## UAM, 2022/2023

### Maitane Gómez González
### Ana Martínez Sabiote

## 1. Sustitución monoalfabeto

## 1.a Método afín
El siguiente programa implementa el método afín.

Llamada a la función:

afin {-C|-D} {-m |Zm|} {-a N×} {-b N+} [-i f ilein] [-o f ileout]


In [1]:
import gmpy2
from gmpy2 import mpz

In [2]:
"""
VERSIÓN SIN GMP 

def algoritmo_euclides(a,b):
    if a%b == 0:
        return b
    else:
        return algoritmo_euclides(b, a%b)
"""

'\nVERSIÓN SIN GMP \n\ndef algoritmo_euclides(a,b):\n    if a%b == 0:\n        return b\n    else:\n        return algoritmo_euclides(b, a%b)\n'

In [3]:
def algoritmo_euclides(a,b):
    if gmpy2.t_mod(a,b) == 0:
        return b
    else:
        return algoritmo_euclides(b, gmpy2.t_mod(a,b))

In [4]:
mcd=algoritmo_euclides(39,150)
print(mcd)

3


In [142]:
algoritmo_euclides(7,15)

mpz(1)

In [5]:
"""
VERSIÓN SIN GMP 

def algoritmo_euclides_extendido(a,b):

    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(b%a, a)
        u=y-(b//a)*x
        v=x
        
    return mcd, u, v
"""

'\nVERSIÓN SIN GMP \n\ndef algoritmo_euclides_extendido(a,b):\n\n    # Identidad de Bézout 1=u*a + v*b\n    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v\n    if a==0:\n        mcd=b\n        u=0\n        v=1\n    else:\n        mcd, x, y = algoritmo_euclides_extendido(b%a, a)\n        u=y-(b//a)*x\n        v=x\n        \n    return mcd, u, v\n'

In [6]:
def algoritmo_euclides_extendido(a,b):
    """
    # Condición a>b, sino las cambiamos
    if b>a:
        aux=a
        a=b
        b=aux
    """
    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(gmpy2.c_mod(b,a), a)
        u=gmpy2.sub(y,(gmpy2.mul(gmpy2.c_div(b,a),x)))
        v=x
        
    return mcd, u, v

In [192]:
def inverso(a,m):
    result = algoritmo_euclides_extendido(a,m)
    # Comprobamos que el mcd es 1 para que exista inverso multiplicativo
    # En consecuencia, a y m determinan una función afín inyectiva
    if result[0] == 1:
        # Entonces devolvemos el coeficiente u (que acompaña a) de la Id. de Bézout
        inv=result[1]
        print(type(result[1]))
        return inv
    else:
        print("Error")

In [193]:
mcd=algoritmo_euclides_extendido(26,15)
print(mcd)

(mpz(1), mpz(-4), mpz(7))


In [194]:
inv=inverso(26,15)
print(inv)

<class 'mpz'>
-4


In [195]:
def read_input(i):
    # Primero tomamos el input de i o de la entrada estándar
    if i==0:
        cadena=input()
    else:
        file=open(i, "r")
        cadena=file.read()
    print("Cadena: {}".format(cadena))
    print(len(cadena))
    return cadena

In [200]:
def afin(modo,m,a,b,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if algoritmo_euclides(a,m) == 1:
        if modo=="-C":
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
            print("Cadena numérica: {}".format(cadena_numerica))
            cadena_cifrada=[]
            for k in cadena_numerica:
                cadena_cifrada.append(((a*k)+b)%m)
            print("Cadena cifrada: {}".format(cadena_cifrada))
        elif modo=="-D":
            cadena_cifrada=read_input(i)
            cadena_cifrada=cadena_cifrada.split(", ")
            cadena_descifrada=[]
            cadena_texto=[]
            inv=inverso(a,m)
            for i in range(len(cadena_cifrada)):
                cadena_cifrada[i]=int(cadena_cifrada[i])
            for k in cadena_cifrada:
                k_descifrado=gmpy2.c_mod(gmpy2.mul((k-b),inv),m)
                if k_descifrado<0:
                    k_descifrado=m+k_descifrado
                cadena_descifrada.append(k_descifrado)
                cadena_texto.append(alfabeto[k_descifrado])
           
            print("Cadena cifrada: {}".format(cadena_cifrada))
            print("Cadena descifrada: {}".format(cadena_descifrada))
            print("Cadena texto: {}".format(cadena_texto))
    else:
        print("{} y {} no son primos relativos. Error".format(a,m))

In [201]:
afin("-C",52,23,3,"cadena.txt")

Cadena: Hola Nueva York!
16
Cadena numérica: [33, 14, 11, 0, 39, 20, 4, 21, 0, 50, 14, 17, 10]
13
Cadena cifrada: [34, 13, 48, 3, 16, 47, 43, 18, 3, 9, 13, 30, 25]
13


In [202]:
afin("-C",130,16,27)

16 y 27 no son primos relativos. Error


In [203]:
afin("-D",52,23,3,"cadena_cifrada.txt")

Cadena: 34, 13, 48, 3, 16, 47, 43, 18, 3, 9, 13, 30, 25
47
<class 'mpz'>
k 
34
K_descifrado
-19
cadena_descifrada
[mpz(33)]
cadena_texto
['H']
k 
13
K_descifrado
-38
cadena_descifrada
[mpz(33), mpz(14)]
cadena_texto
['H', 'o']
k 
48
K_descifrado
-41
cadena_descifrada
[mpz(33), mpz(14), mpz(11)]
cadena_texto
['H', 'o', 'l']
k 
3
K_descifrado
0
cadena_descifrada
[mpz(33), mpz(14), mpz(11), mpz(0)]
cadena_texto
['H', 'o', 'l', 'a']
k 
16
K_descifrado
-13
cadena_descifrada
[mpz(33), mpz(14), mpz(11), mpz(0), mpz(39)]
cadena_texto
['H', 'o', 'l', 'a', 'N']
k 
47
K_descifrado
-32
cadena_descifrada
[mpz(33), mpz(14), mpz(11), mpz(0), mpz(39), mpz(20)]
cadena_texto
['H', 'o', 'l', 'a', 'N', 'u']
k 
43
K_descifrado
-48
cadena_descifrada
[mpz(33), mpz(14), mpz(11), mpz(0), mpz(39), mpz(20), mpz(4)]
cadena_texto
['H', 'o', 'l', 'a', 'N', 'u', 'e']
k 
18
K_descifrado
-31
cadena_descifrada
[mpz(33), mpz(14), mpz(11), mpz(0), mpz(39), mpz(20), mpz(4), mpz(21)]
cadena_texto
['H', 'o', 'l', 'a', 'N', 

In [148]:
inverso(7,15)

<class 'mpz'>


mpz(12)

In [104]:
cadena=read_input("cadena_cifrada.txt")
cadena_dividida=cadena.split()
print(cadena_dividida)
for i in range(len(cadena_dividida)):
    cadena_dividida[i]=int(cadena_dividida[i])
print(cadena_dividida)

Cadena: 10 4 1 3 3 10 7 11 3 1 4 7 0 
29
['10', '4', '1', '3', '3', '10', '7', '11', '3', '1', '4', '7', '0']
[10, 4, 1, 3, 3, 10, 7, 11, 3, 1, 4, 7, 0]


In [182]:
alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
len(alfabeto)
alfabeto[43]

'R'

In [176]:
algoritmo_euclides(23,52)

mpz(1)